In [1]:
#Libraries

import pandas as pd
import numpy as np
from datetime import timedelta


In [ ]:
path ='../data/dataset_flights.csv'

try:
    df_main = pd.read_csv(path)
    print(f"Arquivo '{path}' carregado com sucesso!")
    display(df_main.head())
except FileNotFoundError:
    print(f"Erro: O arquivo não foi encontrado em '{path}'. Verifique o caminho e o nome do arquivo.")
except Exception as e:
    print(f"Ocorreu um erro ao carregar o arquivo: {e}")

Arquivo '../data/dataset_flights.csv' carregado com sucesso!


,Year,Quarter,Month,DayofMonth,DayOfWeek,FlightDate,Tail_Number,Origin,OriginCityName,Dest,...,Diverted,ActualElapsedTime,AirTime,Distance,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,DivAirportLandings
0,2018,2,4,6,5,2018-04-06,N13161,EWR,"Newark, NJ",IND,...,0.0,157.0,111.0,645.0,14.0,0.0,13.0,0.0,9.0,0.0
1,2018,2,4,6,5,2018-04-06,N273JB,ORD,"Chicago, IL",JFK,...,0.0,124.0,101.0,740.0,NaN,NaN,NaN,NaN,NaN,0.0
2,2018,2,4,13,5,2018-04-13,N258NN,ORD,"Chicago, IL",CLE,...,0.0,79.0,46.0,316.0,NaN,NaN,NaN,NaN,NaN,0.0
3,2018,2,4,13,5,2018-04-13,N809UA,IAH,"Houston, TX",DFW,...,0.0,230.0,76.0,224.0,0.0,0.0,142.0,0.0,0.0,0.0
4,2018,2,4,19,4,2018-04-19,N866AS,SFO,"San Francisco, CA",ONT,...,0.0,78.0,55.0,363.0,NaN,NaN,NaN,NaN,NaN,0.0


In [ ]:
import pandas as pd
import numpy as np
from datetime import timedelta

print(f"Initial shape: {df_main.shape}")
print(f"Initial missing values:\n{df_main.isnull().sum()}")

# ============================================
# 1. STANDARDIZE COLUMN HEADERS
# ============================================
df_main.columns = df_main.columns.str.lower().str.strip().str.replace(' ', '_')

# ============================================
# 2. STANDARDIZE STRING CONTENT TO LOWERCASE
# ============================================
str_cols = df_main.select_dtypes(include=['object']).columns
for col in str_cols:
    df_main[col] = df_main[col].str.lower().str.strip()

# ============================================
# 3. CONVERT TO DATETIME
# ============================================
df_main['flightdate'] = pd.to_datetime(df_main['flightdate'])

# ============================================
# 4. REMOVE CANCELLED AND DIVERTED FLIGHTS
# ============================================
print(f"\nRemoving {df_main['cancelled'].sum()} cancelled flights")
print(f"Removing {df_main['diverted'].sum()} diverted flights")

df_main = df_main[
    (df_main['cancelled'] == 0) &
    (df_main['diverted'] == 0)
].copy()

print(f"Shape after removing cancelled/diverted: {df_main.shape}")

# ============================================
# 5. HANDLE MISSING VALUES
# ============================================

# Fill delay breakdown columns (only filled when delayed)
delay_columns = ['carrierdelay', 'weatherdelay', 'nasdelay',
                 'securitydelay', 'lateaircraftdelay']

for col in delay_columns:
    df_main[col] = df_main[col].fillna(0)

# Fill missing delay minutes with 0 (assume on-time if not recorded)
df_main['depdelayminutes'] = df_main['depdelayminutes'].fillna(0)
df_main['arrdelayminutes'] = df_main['arrdelayminutes'].fillna(0)
df_main['arrivaldelaygroups'] = df_main['arrivaldelaygroups'].fillna(0)

# Fill missing elapsed time and airtime with median
df_main['actualelapsedtime'] = df_main['actualelapsedtime'].fillna(
    df_main['actualelapsedtime'].median()
)
df_main['airtime'] = df_main['airtime'].fillna(
    df_main['airtime'].median()
)

print(f"\nMissing values after filling:\n{df_main.isnull().sum()[df_main.isnull().sum() > 0]}")

# ============================================
# 6. DATA QUALITY CHECKS
# ============================================

# Remove negative delays (data errors)
initial_rows = len(df_main)
df_main = df_main[
    (df_main['depdelayminutes'] >= 0) &
    (df_main['arrdelayminutes'] >= 0)
].copy()
print(f"\nRemoved {initial_rows - len(df_main)} rows with negative delays")

# Remove impossible distances
initial_rows = len(df_main)
df_main = df_main[df_main['distance'] > 0].copy()
print(f"Removed {initial_rows - len(df_main)} rows with invalid distance")

# Cap extreme delays at 6 hours (360 minutes)
extreme_delays = (df_main['arrdelayminutes'] > 360).sum()
df_main['arrdelayminutes'] = df_main['arrdelayminutes'].clip(upper=360)
df_main['depdelayminutes'] = df_main['depdelayminutes'].clip(upper=360)
print(f"Capped {extreme_delays} extreme delays at 360 minutes")

# ============================================
# 7. REMOVE DUPLICATES
# ============================================
initial_rows = len(df_main)
df_main = df_main.drop_duplicates()
print(f"Removed {initial_rows - len(df_main)} duplicate rows")

# ============================================
# 8. CREATE ROUTE FEATURES
# ============================================
df_main['flight_route'] = df_main['origincityname'] + ' -> ' + df_main['destcityname']
df_main['airport_route'] = df_main['origin'] + ' -> ' + df_main['dest']

# ============================================
# 9. BASIC TIMESTAMP FEATURES
# ============================================
df_main['day_name'] = df_main['flightdate'].dt.day_name().str.lower()
df_main['is_weekend'] = df_main['flightdate'].dt.dayofweek.isin([5, 6]).astype(int)  # Saturday=5, Sunday=6
df_main['week_of_year'] = df_main['flightdate'].dt.isocalendar().week

# ============================================
# 10. CREATE TARGET VARIABLE (IMPROVED)
# ============================================
# Binary classification: Delay > 15 minutes (industry standard)
df_main['is_delayed'] = (df_main['arrdelayminutes'] > 15).astype(int)

# ============================================
# 11. ADVANCED TEMPORAL FEATURES
# ============================================

# US Federal Holidays function
def get_us_holidays(year):
    """Generate US federal holidays for a given year"""
    holidays = []

    # Fixed holidays
    holidays.append(pd.Timestamp(year, 1, 1))   # New Year's Day
    holidays.append(pd.Timestamp(year, 7, 4))   # Independence Day
    holidays.append(pd.Timestamp(year, 11, 11)) # Veterans Day
    holidays.append(pd.Timestamp(year, 12, 25)) # Christmas

    # MLK Day (3rd Monday in January)
    jan_mondays = pd.date_range(f'{year}-01-01', f'{year}-01-31', freq='W-MON')
    if len(jan_mondays) >= 3:
        holidays.append(jan_mondays[2])

    # Presidents Day (3rd Monday in February)
    feb_mondays = pd.date_range(f'{year}-02-01', f'{year}-02-28', freq='W-MON')
    if len(feb_mondays) >= 3:
        holidays.append(feb_mondays[2])

    # Memorial Day (last Monday in May)
    may_mondays = pd.date_range(f'{year}-05-01', f'{year}-05-31', freq='W-MON')
    if len(may_mondays) > 0:
        holidays.append(may_mondays[-1])

    # Labor Day (1st Monday in September)
    sep_mondays = pd.date_range(f'{year}-09-01', f'{year}-09-30', freq='W-MON')
    if len(sep_mondays) > 0:
        holidays.append(sep_mondays[0])

    # Columbus Day (2nd Monday in October)
    oct_mondays = pd.date_range(f'{year}-10-01', f'{year}-10-31', freq='W-MON')
    if len(oct_mondays) >= 2:
        holidays.append(oct_mondays[1])

    # Thanksgiving (4th Thursday in November)
    nov_thursdays = pd.date_range(f'{year}-11-01', f'{year}-11-30', freq='W-THU')
    if len(nov_thursdays) >= 4:
        holidays.append(nov_thursdays[3])

    return holidays

# Create holiday list for all years in dataset
print("\nGenerating holiday features...")
all_holidays = []
for year in df_main['year'].unique():
    all_holidays.extend(get_us_holidays(year))

all_holidays = set([h.date() for h in all_holidays])

# Holiday features
df_main['is_holiday'] = df_main['flightdate'].dt.date.isin(all_holidays).astype(int)
df_main['days_to_holiday'] = df_main['flightdate'].apply(
    lambda x: min([abs((h - x.date()).days) for h in all_holidays])
)
df_main['is_near_holiday'] = (df_main['days_to_holiday'] <= 3).astype(int)

# Day before/after holiday
df_main['is_day_before_holiday'] = df_main['flightdate'].apply(
    lambda x: (x.date() + timedelta(days=1)) in all_holidays
).astype(int)

df_main['is_day_after_holiday'] = df_main['flightdate'].apply(
    lambda x: (x.date() - timedelta(days=1)) in all_holidays
).astype(int)

# Season features
df_main['is_summer_travel'] = df_main['month'].isin([6, 7, 8]).astype(int)
df_main['is_winter_travel'] = df_main['month'].isin([12, 1]).astype(int)
df_main['is_spring_break'] = df_main['month'].isin([3, 4]).astype(int)
df_main['is_thanksgiving_week'] = (
    (df_main['month'] == 11) & (df_main['week_of_year'].isin([47, 48]))
).astype(int)
df_main['is_christmas_week'] = (
    (df_main['month'] == 12) & (df_main['dayofmonth'] >= 20)
).astype(int)

# Cyclical encoding for temporal features
df_main['month_sin'] = np.sin(2 * np.pi * df_main['month'] / 12)
df_main['month_cos'] = np.cos(2 * np.pi * df_main['month'] / 12)
df_main['dayofweek_sin'] = np.sin(2 * np.pi * df_main['dayofweek'] / 7)
df_main['dayofweek_cos'] = np.cos(2 * np.pi * df_main['dayofweek'] / 7)

print(f"Holidays identified: {df_main['is_holiday'].sum()}")

# ============================================
# 12. DISTANCE & ROUTE FEATURES
# ============================================

# Distance categories
df_main['distance_category'] = pd.cut(
    df_main['distance'],
    bins=[0, 500, 1500, 3000, 10000],
    labels=['short', 'medium', 'long', 'ultra_long']
)

# Transcontinental flag
df_main['is_transcontinental'] = (df_main['distance'] > 2000).astype(int)

# ============================================
# 13. FINAL CLEANUP
# ============================================

# Sort by date for temporal splits
df_main = df_main.sort_values('flightdate').reset_index(drop=True)

# Ensure all column names are clean
df_main.columns = df_main.columns.str.strip().str.lower().str.replace(' ', '_')

# ============================================
# 14. SUMMARY
# ============================================
print("\n" + "="*60)
print("DATA CLEANING SUMMARY")
print("="*60)
print(f"Final shape: {df_main.shape}")
print(f"\nTarget distribution:")
print(df_main['is_delayed'].value_counts())
print(f"Delay rate: {df_main['is_delayed'].mean():.2%}")
print(f"\nDate range: {df_main['flightdate'].min()} to {df_main['flightdate'].max()}")
print(f"Years: {sorted(df_main['year'].unique())}")
print(f"\nMissing values remaining:")
print(df_main.isnull().sum()[df_main.isnull().sum() > 0])
print("="*60)

# Display sample
print("\nFirst few rows after cleaning:")
df_main.head()